In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers


In [3]:
!pip install nltk

In [4]:
!pip install torch


In [5]:
import pandas as pd
from transformers import LongformerModel, AutoTokenizer
import torch as pt
from sklearn.model_selection import train_test_split
import os
import numpy as np
import chardet
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import nltk
import re
from nltk.corpus import stopwords
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader, TensorDataset

In [6]:
path = "/content/drive/MyDrive/Colab Notebooks/code smells tesi/projects"
projects = []
code_smells = ['CDSBP', 'CC', 'LC', 'LZC', 'RB', 'SC']
# Itera su tutti i file e le cartelle nella cartella specificata
for item in os.listdir(path):
    # Se l'elemento nella cartella è una cartella, aggiungi il nome alla lista
    if os.path.isdir(os.path.join(path, item)):
       projects.append(item)
    # Stampa la lista di nomi dei progetti
print(projects)

['ant-ivy-2.0.0-alpha2', 'ant-rel-1.8.3', 'cassandra-cassandra-1.0.0', 'elasticsearch-v0.19.0', 'hadoop-release-0.6.0', 'hive-release-0.9.0', 'hsqldb-2.2.8', 'karaf-karaf-2.3.0', 'lucene-releases-lucene-solr-3.6.0', 'manifold-cf-release-0.6', 'nutch-release-1.4', 'pig-release-0.8.0', 'qpid-0.14', 'struts-STRUTS_2_3_4', 'xerces2-j-Xerces-J_2_3_0']


In [7]:
projects = ['ant-ivy-2.0.0-alpha2', 'ant-rel-1.8.3', 'cassandra-cassandra-1.0.0', 'elasticsearch-v0.19.0', 'hadoop-release-0.6.0', 'hive-release-0.9.0', 'hsqldb-2.2.8', 'karaf-karaf-2.3.0']

In [8]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/code smells tesi/Code_Smells_NLP-Finale/dataset/ultimate_dataset.csv")

In [9]:
df

,Project_name,Component,CBO,CYCLO,DIT,ELOC,FanIn,FanOut,LCOM,LOC,...,WLOCNAMM,WMC,WMCNAMM,NMNOPARAM,CDSBP,CC,LC,LZC,RB,SC
0,ant-ivy-2.0.0-alpha2,/*\n * Licensed to the Apache Software Founda...,41.0,136.0,1.0,346.0,2036.0,83.0,1482.0,346.0,...,5.592593,136.0,0.0,23.0,0,0,0,0,0,0
1,ant-ivy-2.0.0-alpha2,/*\n * Licensed to the Apache Software Founda...,31.0,172.0,1.0,221.0,0.0,207.0,0.0,221.0,...,3.583333,172.0,0.0,6.0,0,0,0,0,0,0
2,ant-ivy-2.0.0-alpha2,/*\n * Licensed to the Apache Software Founda...,43.0,35.0,1.0,295.0,20.0,121.0,45.0,295.0,...,28.900000,35.0,0.0,2.0,0,0,0,0,0,0
3,ant-ivy-2.0.0-alpha2,/*\n * Licensed to the Apache Software Founda...,8.0,25.0,2.0,59.0,0.0,11.0,0.0,59.0,...,4.363636,25.0,0.0,5.0,0,0,0,0,0,0
4,ant-ivy-2.0.0-alpha2,/*\n * Licensed to the Apache Software Founda...,16.0,26.0,2.0,101.0,2.0,37.0,44.0,101.0,...,8.888889,26.0,0.0,4.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8139,xerces2-j-Xerces-J_2_3_0,"/*\n * The Apache Software License, Version 1....",4.0,25.0,2.0,55.0,5.0,15.0,28.0,55.0,...,9.375000,25.0,0.0,5.0,0,0,0,0,0,0
8140,xerces2-j-Xerces-J_2_3_0,// XMLFilterImpl.java - base SAX2 filter imple...,14.0,120.0,1.0,449.0,2.0,29.0,364.0,449.0,...,12.548387,120.0,0.0,9.0,0,0,0,0,0,0
8141,xerces2-j-Xerces-J_2_3_0,// XMLReaderAdapter.java - adapt an SAX2 XMLRe...,15.0,45.0,1.0,279.0,2.0,20.0,115.0,279.0,...,9.227273,45.0,0.0,4.0,0,0,0,0,0,0
8142,xerces2-j-Xerces-J_2_3_0,// XMLReaderFactory.java - factory for creatin...,5.0,16.0,1.0,97.0,11.0,12.0,6.0,97.0,...,22.000000,16.0,0.0,2.0,0,0,0,0,0,1


In [10]:
c = 0
for x in df['Component']:
  parole = x.split()
  if(len(parole)>1024):
    c +=1
print(c)

2652


In [11]:
# nltk.download('stopwords')

# stop_words = set(stopwords.words('english'))

# def remove_comments(code):
#     # Rimuove i commenti su una singola riga
#     code = re.sub(r'//.*', '', code)
#     # Rimuove i commenti su più righe
#     code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
#     # Rimuove le stopwords
#     tokens = [word.lower() for word in code.split() if word.lower() not in stop_words]
#     filtered_text = ' '.join(tokens)
#     # Rimuoviamo gli import
#     code = re.sub(r'import\s+.*?;', '', code, flags=re.DOTALL)
#     # Rimuoviamo i package
#     code = re.sub(r'package\s+.*?;', '', code, flags=re.DOTALL)
#     #Rimuove \n e \t
#     code = re.sub(r'[\n\t]', '', code)
#     return code

# for i in range(len(df)):
#     df['Component'][i] = remove_comments(df['Component'][i])

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

# Aggiungi stopwords specifiche per Java
java_stopwords = [
    'java', 'protected', 'class', 'interface', 'package',
    'import', 'static', 'void', 'new', 'return', 'extends', 'implements', 'null',
    'try', 'catch', 'finally', 'throw', 'throws'
]

stop_words.update(java_stopwords)

def remove_comments(code):
    # Rimuove i commenti su una singola riga
    code = re.sub(r'//.*', '', code)
    # Rimuove i commenti su più righe
    code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
    # Rimuoviamo gli import
    code = re.sub(r'import\s+.*?;', '', code, flags=re.DOTALL)
    # Rimuoviamo i package
    code = re.sub(r'package\s+.*?;', '', code, flags=re.DOTALL)
    # Rimuove \n e \t
    code = re.sub(r'[\n\t]', '', code)
    # Rimuove le stopwords solo se sono parole singole
    tokens = [word.lower() for word in code.split() if word.lower() not in stop_words]
    filtered_text = ' '.join(tokens)
    return filtered_text

for i in range(len(df)):
    df['Component'][i] = remove_comments(df['Component'][i])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-11-0bbbb70b7342>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Component'][i] = remove_comments(df['Component'][i])


In [12]:
df.dropna(inplace=True)

In [13]:
c = 0
for x in df['Component']:
  parole = x.split()
  if(len(parole)>1536):
    c +=1
print(c)

1022


In [14]:
import pandas as pd
import random

def create_filtered_dataframe(input_dataframe, required_cols, num_instances=4000):
    # Filtra le istanze con almeno un 1 nelle colonne richieste
    filtered_df = input_dataframe[input_dataframe[required_cols].sum(axis=1) >= 1].copy()
    # Verifica e rimuovi eventuali duplicati
    filtered_df.drop_duplicates(inplace=True)
    # Seleziona casualmente altre istanze con tutte le colonne a 0 senza duplicati
    remaining_instances = num_instances - len(filtered_df)
    if remaining_instances > 0:
        zero_instances = input_dataframe[input_dataframe[required_cols].sum(axis=1) == 0]
        if len(zero_instances) < remaining_instances:
            raise ValueError("Non ci sono abbastanza istanze con tutte le colonne a 0.")

        # Rimuovi eventuali duplicati tra le istanze con tutte le colonne a 0
        zero_instances = zero_instances.drop_duplicates()

        # Seleziona casualmente le istanze con tutte le colonne a 0
        random_zero_instances = zero_instances.sample(remaining_instances)
        filtered_df = pd.concat([filtered_df, random_zero_instances])

    # Riduci il dataframe alle prime 4000 istanze, se necessario
    if len(filtered_df) > num_instances:
        filtered_df = filtered_df.sample(num_instances)

    # Reimposta gli indici in modo sequenziale a partire da 1
    filtered_df.reset_index(drop=True, inplace=True)

    return filtered_df

# Esempio di utilizzo:
# Supponiamo che il tuo dataframe originale si chiami 'df'
required_columns = ['CDSBP', 'CC', 'LC', 'LZC', 'RB', 'SC']
dataframe = create_filtered_dataframe(df, required_columns)


In [15]:
dataframe

,Project_name,Component,CBO,CYCLO,DIT,ELOC,FanIn,FanOut,LCOM,LOC,...,WLOCNAMM,WMC,WMCNAMM,NMNOPARAM,CDSBP,CC,LC,LZC,RB,SC
0,ant-ivy-2.0.0-alpha2,public ivybuildlist ivytask { private list bui...,26.0,68.0,2.0,306.0,21.0,96.0,389.0,306.0,...,10.840000,68.0,0.0,12.0,0,0,0,0,0,1
1,ant-ivy-2.0.0-alpha2,public abstract ivypostresolvetask ivytask { p...,14.0,69.0,2.0,258.0,25.0,85.0,526.0,258.0,...,7.851852,69.0,0.0,18.0,0,0,0,0,0,1
2,ant-ivy-2.0.0-alpha2,public ivypublish ivytask { private string org...,19.0,75.0,2.0,313.0,4.0,64.0,769.0,313.0,...,5.727273,75.0,0.0,20.0,0,0,0,0,0,1
3,ant-ivy-2.0.0-alpha2,public ivyresolve ivytask { private file file ...,14.0,62.0,2.0,240.0,115.0,52.0,583.0,240.0,...,6.714286,62.0,0.0,18.0,0,0,0,0,0,1
4,ant-ivy-2.0.0-alpha2,public ivynode comparable { private final patt...,37.0,229.0,1.0,834.0,194.0,317.0,3030.0,834.0,...,9.180723,229.0,0.0,31.0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,elasticsearch-v0.19.0,public deleteindextemplateresponse actionrespo...,5.0,10.0,1.0,23.0,18.0,3.0,0.0,23.0,...,2.833333,10.0,0.0,3.0,0,0,0,0,0,0
3996,elasticsearch-v0.19.0,public validateactions { public actionrequestv...,0.0,3.0,1.0,11.0,10.0,1.0,0.0,11.0,...,7.000000,3.0,0.0,0.0,0,0,0,0,0,0
3997,ant-rel-1.8.3,public leadpipeinputstream pipedinputstream { ...,6.0,26.0,2.0,117.0,4.0,9.0,34.0,117.0,...,12.111111,26.0,0.0,2.0,0,0,0,0,0,0
3998,ant-ivy-2.0.0-alpha2,public latestconflictmanager abstractconflictm...,10.0,23.0,2.0,96.0,13.0,26.0,12.0,96.0,...,7.666667,23.0,0.0,3.0,0,0,0,0,0,0


In [16]:
import torch
import torch.nn as nn
from transformers import AdamW

code_smells = ['CC', 'LC']

# Dataframe per i risultati finali
CB_final_df = pd.DataFrame()

# Dizionario per salvare le predizioni
CB_predictions = {}

# Dizionario per salvare le metriche
CB_metrics = {}

# Loop attraverso i progetti
for project in projects:
    print("Current project:", project)

    # Caricamento dei dati specifici del progetto
    project_df = dataframe[dataframe['Project_name'] == project]
    if len(project_df) == 0:
        continue

    # Inizializza i dizionari per il progetto corrente

    CB_predictions[project] ={}
    CB_metrics[project] ={}

    # Loop attraverso i code smell
    for smell in code_smells:
        print("Current code smell:", smell)

        X = project_df.drop(columns=['CDSBP', 'CC', 'LC', 'LZC', 'RB', 'SC'])
        y = project_df[str(smell)]

        X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=1)

        ##
        c = 0
        for x in Y_test:
          if(x == 1):
            c += 1
        if(c < 2):
          continue

        #Dati per CodeBert
        X_train_comp = X_train['Component']
        X_test_comp = X_test['Component']

        # Fitting vari modelli

        # Caricamento del tokenizer e del modello pre-addestrato CodeBERT
        tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")
        model = AutoModelForSequenceClassification.from_pretrained('allenai/longformer-base-4096', num_labels=1)

        # Tokenizzazione del set di addestramento
        train_encodings = tokenizer(list(X_train_comp), truncation=True, padding=True, max_length = 1024)
        train_labels = pt.tensor(Y_train.values)
        # Tokenizzazione del set di test
        test_encodings = tokenizer(list(X_test_comp), truncation=True, padding=True, max_length = 1024)
        test_labels = pt.tensor(Y_test.values)

        # Inizio training


        # Imposta la dimensione del batch desiderata
        batch_size = 5

        # Definizione del dispositivo (GPU o CPU)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Conversione dei dati tokenizzati in tensori PyTorch
        train_inputs = torch.tensor(train_encodings['input_ids']).to(device)
        train_labels = torch.tensor(Y_train.values).unsqueeze(1).to(device)

        val_inputs = torch.tensor(test_encodings['input_ids']).to(device)
        val_labels = torch.tensor(Y_test.values).unsqueeze(1).to(device)

        # Creazione del dataset di addestramento
        train_dataset = TensorDataset(train_inputs, train_labels)

        # Creazione del dataset di validazione
        val_dataset = TensorDataset(val_inputs, val_labels)

        # Crea i dataloader con la nuova batch_size
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

        # Creazione dell'ottimizzatore
        optimizer = AdamW(model.parameters(), lr=1e-5)

        # Definizione della funzione di perdita
        loss_function = loss_function = nn.BCEWithLogitsLoss()

        # Addestramento del modello
        model.to(device)
        model.train()

        for epoch in range(3):
            for batch in train_dataloader:
                optimizer.zero_grad()

                inputs = batch[0].to(device)
                labels = batch[1].to(device)
                labels = labels.float()

                outputs = model(inputs)
                logits = outputs.logits

                loss = loss_function(logits.squeeze(), labels.squeeze())  # Calcola la loss tra logits e etichette
                loss.backward()
                optimizer.step()

                #print(f"Epoch {epoch+1} - Loss: {loss.item()}")
                # Libera la memoria GPU
                del inputs, labels
                torch.cuda.empty_cache()

        # Fine training
        predictions = []
        torch.cuda.empty_cache()
        for batch in val_dataloader:
            inputs = batch[0].to(device)
            labels = batch[1].to(device)

            val_outputs = model(inputs)
            val_predicted_labels = (val_outputs.logits.sigmoid().squeeze() > 0.5).int()

            if val_predicted_labels.dim() > 0:  # Verifica la dimensione del tensore
              predictions.extend(val_predicted_labels.cpu().numpy())

            del inputs, labels, val_outputs, val_predicted_labels
            torch.cuda.empty_cache()

        predictions = np.array(predictions)
        test_labels = test_labels.cpu().numpy()

        precision = precision_score(test_labels, predictions, average='macro')
        accuracy = accuracy_score(test_labels, predictions)
        mcc = matthews_corrcoef(test_labels, predictions)
        recall = recall_score(test_labels, predictions, average='macro')
        f1 = f1_score(test_labels, predictions, average='macro')
        print("Precision",precision)
        print("F1", f1)
        print("MCC", mcc)
        CB_metrics[project][smell] = {
            "Precision": precision,
            "Accuracy": accuracy,
            "Recall": recall,
            "F1": f1,
            "MCC": mcc
        }

        CB_predictions[project][smell] = predictions

        del predictions
        torch.cuda.empty_cache()


        CB_temp_pred = pd.DataFrame({
            'Project_name': [project],
            'Smell': [smell],
            'Y_pred': [CB_predictions[project][smell].tolist()],
            'Y_test': [Y_test.tolist()],
            'precision': precision,
            'accuracy': accuracy,
            'recall': recall,
            'f1': f1,
            'mcc': mcc,
        })

        CB_final_df = pd.concat([CB_final_df, CB_temp_pred], ignore_index=True)

CB_final_df.to_csv("/content/drive/MyDrive/Colab Notebooks/code smells tesi/Code_Smells_NLP-Finale/LF_results.csv", index=False)

Current project: ant-ivy-2.0.0-alpha2
Current code smell: CC
Current code smell: LC


Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision 0.4838709677419355
F1 0.4918032786885246
MCC 0.0
Current project: ant-rel-1.8.3
Current code smell: CC
Current code smell: LC


Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision 0.49473684210526314
F1 0.4973544973544973
MCC 0.0
Current project: cassandra-cassandra-1.0.0
Current code smell: CC
Current code smell: LC


...

Precision 0.4787234042553192
F1 0.4891304347826087
MCC 0.0
Current project: hive-release-0.9.0
Current project: hsqldb-2.2.8
Current code smell: CC


Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision 0.491869918699187
F1 0.4959016393442623
MCC 0.0
Current code smell: LC


Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


OutOfMemoryError: ignored

In [ ]:
# import torch
# from transformers import AdamW
# from torch.utils.data import DataLoader, TensorDataset
# import torch.nn as nn
# from sklearn.metrics import accuracy_score, precision_score, jaccard_score, hamming_loss
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.metrics import multilabel_confusion_matrix

# code_smells = ['CDSBP','CC','LC','LZC','RB','SC']

# for smell in code_smells:

#   # Caricamento del tokenizer e del modello pre-addestrato CodeBERT
#   model = LongformerModel.from_pretrained("allenai/longformer-base-4096", num_labels=1)
#   tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

#   # Creazione X e y. X = dataset senza le etichette, y = etichette
#   dataframe2 = dataframe.drop(columns=['CDSBP','CC','LC','LZC','RB','SC'])
#   y = dataframe[str(smell)]

#   # Divisione in train, validation e test set
#   X_train, X_val_test, Y_train, Y_val_test = train_test_split(dataframe2, y, test_size=0.4, random_state=1)
#   X_val, X_test, Y_val, Y_test = train_test_split(X_val_test, Y_val_test, test_size=0.5, random_state=1)
#   # Tokenizzazione del set di addestramento
#   train_encodings = tokenizer(list(X_train['Component']), truncation=True, padding=True, max_length = 1024)
#   train_labels = pt.tensor(Y_train.values)

#   # Tokenizzazione del set di validazione
#   val_encodings = tokenizer(list(X_val['Component']), truncation=True, padding=True, max_length = 1024)
#   val_labels = pt.tensor(Y_val.values)

#   # Tokenizzazione del set di test
#   test_encodings = tokenizer(list(X_test['Component']), truncation=True, padding=True, max_length = 1024)
#   test_labels = pt.tensor(Y_test.values)


#   device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#   # Creazione del dataset di addestramento e validazione
#   train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']), train_labels)
#   val_dataset = TensorDataset(torch.tensor(val_encodings['input_ids']), val_labels)

#   # Creazione dei dataloader
#   batch_size = 4
#   train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#   val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

#   # Creazione del modello
#   model.to(device)
#   model.train()

#   # Impostazione dell'ottimizzatore
#   optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

#   # Definizione della funzione di perdita
#   loss_function = nn.CrossEntropyLoss()

#   # Addestramento del modello
#   for epoch in range(3):
#       for batch in train_dataloader:
#           optimizer.zero_grad()
#           inputs = batch[0].to(device)
#           labels = batch[1].to(device)

#           outputs = model(inputs)
#           logits = outputs.last_hidden_state[:, 0, :]

#           loss = loss_function(logits, labels)
#           loss.backward()
#           optimizer.step()

#           print(f"Epoch {epoch+1} - Loss: {loss.item()}")

#           # Liberazione memoria GPU
#           del inputs, labels
#           torch.cuda.empty_cache()



#   val_accuracy = 0
#   precision_avg = 0
#   jaccard_score_avg = 0
#   hamming_loss_score = 0
#   total_val_samples = len(val_dataloader.dataset)

#   with torch.no_grad():
#       for batch in val_dataloader:
#           inputs = batch[0].to(device)
#           labels = batch[1].to(device)

#           val_outputs = model(inputs)
#           # print(val_outputs)
#           val_predicted_labels = torch.argmax(val_outputs.pooler_output, dim=1)



#           val_accuracy = accuracy_score(labels.cpu().numpy(), val_predicted_labels.cpu().numpy())
#           precision_avg = precision_score(labels.cpu().numpy(), val_predicted_labels.cpu().numpy(), average='macro')


#           jaccard_score_avg += jaccard_score(labels.cpu().numpy(), val_predicted_labels.cpu().numpy(), average='macro')
#           hamming_loss_score += hamming_loss(labels.cpu().numpy(), val_predicted_labels.cpu().numpy())

#           # Libera la memoria GPU
#           del inputs, labels, val_outputs, val_predicted_labels
#           torch.cuda.empty_cache()

#   val_accuracy /= total_val_samples
#   precision_avg /= total_val_samples
#   jaccard_score_avg /= total_val_samples
#   hamming_loss_score /= total_val_samples

#   print(f"Validation Accuracy: {val_accuracy}")
#   print(f"Precision: {precision_avg}")
#   print(f"Jaccard Score: {jaccard_score_avg}")
#   print(f"Hamming Loss: {hamming_loss_score}")



#   # Calcola le predizioni del modello sul dataset di validazione
#   model.eval()
#   with torch.no_grad():
#       predicted_labels = []
#       true_labels = []
#       for batch in val_dataloader:
#           inputs = batch[0].to(device)
#           labels = batch[1].to(device)
#           outputs = model(inputs)
#           predicted_labels.extend(outputs.logits.sigmoid().round().cpu().numpy())
#           true_labels.extend(labels.cpu().numpy())

#   # Calcola la matrice di confusione
#   confusion_matrix = multilabel_confusion_matrix(true_labels, predicted_labels)

#   # Etichette delle classi
#   class_labels = ['CDSBP', 'CC', 'LC', 'LZC', 'RB', 'SC']

#   # Visualizza la matrice di confusione
#   plt.figure(figsize=(10, 8))
#   for i, cm in enumerate(confusion_matrix):
#       plt.subplot(2, 3, i + 1)
#       sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
#       plt.xlabel('Etichetta predetta')
#       plt.ylabel('Etichetta reale')
#       plt.title(f'Matrice di confusione - {class_labels[i]}')
#   plt.tight_layout()
#   plt.show()